## Setup
All the imports. simutils contains a set of functions I created for generating synthetic data for these experiments.

In [30]:
#!pip install statsmodels
#!pip install scikit-learn
#!pip install numpy
#!pip install pandas
#!pip install matplotlib
#!pip install scipy

import simutils as sim
import numpy as np
import pandas as pd

## Predictor Variables of Non-Interest
First, create some synthetic age, credit score and arbitrary hypothetically relevant personality characteristic values. Most of these variables are nuisance or confounding variables, but will partially determine both the predictor of interest and the dependent variable.

Age will be normally distributed (M=35, SD=10) and range from 18 to 78. Credit scores are 700-730 on average, but tend to be lower for younger people, and are reduced by up to 40 points for the youngest, relative to the oldest. The hypothetical personality variable will reflect whatever set of intrinsic characteristics (e.g., inquisitiveness, intelligence) that are normally distributed and might drive the behaviour of interest.

In [31]:
N=10000
split=int(N/2)
ages=sim.generate_ages(35, 10, N)
credit_scores=sim.generate_credit_scores(ages)
personality=np.random.normal(loc=0, scale=1, size=N)
#create data frame out of factors
Z=pd.DataFrame({
    'age': ages,
    'credit_score': credit_scores,
    'personality': personality
    })
train_Z=Z.iloc[:split,:]
test_Z=Z.iloc[split:,:]


## Predictor of Interest
The independent measure will be the number of recorded interactions with product information pages, obtained from cookies, since monitoring was implemented.

Our goal will be to determine the causal relationship between clicking behavior and the dependent variable, personal wealth. This relationship can be quantified by computing the coefficient or weight in a regression or machine learning model that predicts wealth from clicks.

Clicking behavior will be influenced by all three of the variables in Z. These influences will be non-linear, which would make a linear regression-based approach a poor choice, though the number of clicks will be a linear combination of all three influences. I will explicitly determine each confound's influence on an individual's clicking behavior, making it easier to establish the ground-truth.

In [32]:
from scipy.stats import randint

clicks=sim.generate_clicks(Z)
clicks.columns=['age_delta', 'csv_delta', 'personality_delta']
#external random factors are responsible for the baseline number of interactions
clicks['baseline'] = randint.rvs(10,50, size=N)
clicks['total'] = clicks.apply(np.sum, axis=1)
train_clicks=clicks.iloc[:split,:]
test_clicks=clicks.iloc[split,:]
train_clicks.head()

,age_delta,csv_delta,personality_delta,baseline,total
0,43,6,16,15,80
1,37,34,11,29,111
2,70,11,-16,22,87
3,63,16,-2,41,118
4,-3,45,12,43,97


From the perspective of an all-knowing oracle, we can isolate the total number of clicks for each person that were not attributable to either age or credit score value:

In [33]:
clean_clicks=pd.DataFrame(clicks['personality_delta']+clicks['baseline'])
clean_clicks.columns=['i_total']

## The Dependent Variable: Wealth/Value
Now to create the dependent variable, portfolio value, we can use a similar procedure. The DV value will be caused in part by the number of clicks, as well as age and credit score and we can see whether we can tease apart these components. **Every click causes an increases a person's wealth by $500**, and this is the causal parameter we are trying to discover.

In [34]:
#baseline wealth increases as individuals get older until ~75, at which point it drops off
age_wealth=Z.apply(lambda row: sim.age_worth(row[0]), axis=1)
#baseline wealth increases for people with better credit scores, who can get better rates
csv_wealth=Z.apply(lambda row: sim.csv_worth(row[1]), axis=1)
#wealth increases caused by the behavior we care about (clicks), itself influenced by age and credit
click_wealth=clicks.apply(lambda row: sim.clicks_worth(row[4]), axis=1)
#wealth attributable to all the other random chance factors
circumstance_wealth=clicks.apply(lambda row: sim.circumstance_worth(), axis=1)
wealth=pd.concat([age_wealth, csv_wealth, click_wealth, circumstance_wealth], axis=1)
wealth.columns=['age_wealth', 'csv_wealth', 'click_wealth', 'circumstance_wealth']
wealth["total_wealth"]=wealth.apply(np.sum, axis=1)
train_wealth=wealth.iloc[:split,:]
test_wealth=wealth.iloc[split:,:]
train_wealth.head()


,age_wealth,csv_wealth,click_wealth,circumstance_wealth,total_wealth
0,18379,54368,40008,40969,153724
1,15229,39603,55490,36873,147195
2,35530,60454,43501,37761,177246
3,38973,63080,59001,42377,203431
4,11037,32217,48501,29395,121150


Like for clicks, I want the oracle measure of the wealth attributable to those clicks that were not driven by age and credit score:

In [35]:
clean_wealth=clean_clicks.apply(lambda row: sim.clicks_worth(row[0]), axis=1)

## Naive Multiple Regression
So the goal is to use the customer's interaction data ("clicks") to explain the value of their portfolio ("total wealth"). In this scenario, we have reason to believe that age and credit score affect both the number of interactions and the customer's total wealth and need to remove these confounding effects. Before we do so, let's run a naive multiple regression, paying attention to the coefficient for the clicks variable, to see the impact of the confounding variables on our parameter estimate.

In [36]:
import statsmodels.formula.api as smf
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_predict

y=wealth["total_wealth"]
x=clicks["total"]
basedf=pd.DataFrame({'y': y, 'clicks': x})
basedf["age"]=Z["age"]
basedf["csv"]=Z["credit_score"]

naive_model = smf.ols(formula='y ~ 1 + clicks + age + csv', data = basedf).fit()
print(naive_model.summary())


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.904
Model:                            OLS   Adj. R-squared:                  0.904
Method:                 Least Squares   F-statistic:                 3.154e+04
Date:                Thu, 16 Nov 2023   Prob (F-statistic):               0.00
Time:                        13:20:14   Log-Likelihood:            -1.0645e+05
No. Observations:               10000   AIC:                         2.129e+05
Df Residuals:                    9996   BIC:                         2.129e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.132e+05   6111.158     18.530      0.0

In the above model summary, the coefficient estimate for clicks is quite biased: we know the true value to be 500, so the model is greatly overestimating the impact of clicking. There's also an indicator of potential multicollinearity, which is not at all surprising, since age drives csv, and age and csv drive the number of clicks.

## Double Machine Learning
Now let's use double machine learning to try to remove the confounding effects of age and credit score. This is done by using the confounders to predict the variables we care about (clicks and wealth), and calculating the residuals. The residualized values are the components of the variables of interest that couldn't be adequately predicted by the confounders, and are orthogonal with respect to the confounding variables.

The particular ML algorithm used isn't critical, as long as it is appropriate for your variables. Here, I am predicting continuous integer values, and had never before had an opportunity to try out a boosted decision tree algorithm. For categorical data, an SVM or even neural network might be appropriate.

In [37]:
M_clicks=GradientBoostingRegressor()
M_wealth=GradientBoostingRegressor()

#use cross-validation values to obtain residuals
residualized_y = y-cross_val_predict(M_wealth, Z[["age", "credit_score"]], y, cv=3)
residualized_x = x-cross_val_predict(M_clicks, Z[["age", "credit_score"]], x, cv=3)
df=pd.DataFrame()
df["Y_hat"]=residualized_y
df["X_hat"]=residualized_x
df["clean_X"]=clean_clicks
df["dirty_Y"]=wealth["click_wealth"]
df["clean_Y"]=clean_wealth
df["age"]=Z["age"]
df["credit_score"]=Z["credit_score"]

train_df=df.iloc[:split,:]
test_df=df.iloc[split:,:]

#fit the residualized data and check out the model prediction
DML_model = smf.ols(formula='Y_hat ~ 1 + X_hat', data = train_df).fit()
print(DML_model.summary())


                            OLS Regression Results                            
Dep. Variable:                  Y_hat   R-squared:                       0.711
Model:                            OLS   Adj. R-squared:                  0.711
Method:                 Least Squares   F-statistic:                 1.229e+04
Date:                Thu, 16 Nov 2023   Prob (F-statistic):               0.00
Time:                        13:20:15   Log-Likelihood:                -49646.
No. Observations:                5000   AIC:                         9.930e+04
Df Residuals:                    4998   BIC:                         9.931e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -84.5802     70.231     -1.204      0.2

## Oracle Models
The double-ML model using the orthogonalized values are much closer to the true parameter value.

From the perspective of an all-knowing oracle, we can compare the double-ML results against models using the ground-truth oracle data. In my first attempt, I modeled wealth values that were computed only using the total number of clicks (which itself was influenced by age and credit score), which I then realized doesn't quite capture what double-ML was supposed to have done:

In [42]:
#fit the number of clicks unrelated to age/credit score to the wealth attributable to clicks alone
Oracle_model_1 = smf.ols(formula='dirty_Y ~ 1 + clean_X', data = train_df).fit()
print(Oracle_model_1.summary())


                            OLS Regression Results                            
Dep. Variable:                dirty_Y   R-squared:                       0.475
Model:                            OLS   Adj. R-squared:                  0.475
Method:                 Least Squares   F-statistic:                     4521.
Date:                Thu, 16 Nov 2023   Prob (F-statistic):               0.00
Time:                        13:20:44   Log-Likelihood:                -51874.
No. Observations:                5000   AIC:                         1.038e+05
Df Residuals:                    4998   BIC:                         1.038e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   3.207e+04    307.405    104.326      0.0

I followed up by repeating the model, using the click behavior not attributable to age and credit score to generate the wealth attributable to the clicking behavior that is also not attributable to age and credit score:

In [43]:
#fit the number of clicks unrelated to age/credit score to the wealth attributable to clicks alone
Oracle_model_2 = smf.ols(formula='clean_Y ~ 1 + clean_X', data = train_df).fit()
print(Oracle_model_2.summary())

                            OLS Regression Results                            
Dep. Variable:                clean_Y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 8.394e+09
Date:                Thu, 16 Nov 2023   Prob (F-statistic):               0.00
Time:                        13:21:06   Log-Likelihood:                -15818.
No. Observations:                5000   AIC:                         3.164e+04
Df Residuals:                    4998   BIC:                         3.165e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.9067      0.227     -3.995      0.0

## Comparing Orthogonalized Scores to Oracle Values
A final question to consider is whether the orthogonalized scores are equivalent to "clean" data; that is, whether they map to the ground-truth oracle data without the influence of the nuisance variables. Consider that we have access to the components of each person's click behaviour that are independent of age and credit score. We can quantify the correspondence of these values with the residualized scores. 

In [44]:
from scipy import stats
import matplotlib.pyplot as plt

res=stats.spearmanr(residualized_x, clean_clicks)
bad=stats.spearmanr(residualized_x, clicks["total"])
print(f"The residualized values are correlated at {int(res.statistic*100)}% with the true values and {int(bad.statistic *100)}% with the dirty values")

The residualized values are correlated at 94% with the true values and 71% with the dirty values


## Conditional Average Treatment Effects (CATE)
Trying to get my head wrapped around how CATE is implemented. In all the examples I have seen, after the residuals are calculated, CATE is computed by predicting Y_hat using the treatment * nuisance variables. This strikes me as bizarre, since the nuisance variables are theoretically unrelated to the residualized values, and thus all regression coefficients should be zero (they won't be exactly zero, but that's because of error).


In [54]:

#fit the residualized data and check out the model prediction
cate_model = smf.ols(formula='Y_hat ~ X_hat * (age + credit_score)', data = train_df).fit()

cate_test = test_df.assign(cate=cate_model.predict(test_df.assign(X_hat=1))
                        - cate_model.predict(test_df.assign(X_hat=0)))
cate_test.head()


,Y_hat,X_hat,clean_X,dirty_Y,clean_Y,age,credit_score,cate
5000,1340.155037,9.800987,42,53002,20999,32,683,497.380323
5001,-6430.844963,-20.199013,20,37998,9998,32,681,496.237540
5002,-12169.927278,-24.587529,14,39995,6994,28,654,494.702142
5003,-5097.596630,-6.602138,33,36007,16501,39,729,499.353031
5004,-4371.321940,-18.287082,24,34497,11991,37,716,498.871028
